In [96]:
import string
import pandas as pd
import numpy as np
import requests

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)

liste_salaires = df['lieu'].apply(lambda x : x[1] if len(x) > 1 else '')

df['lieu'] = df['lieu'].apply(lambda x : x[0])
df['Nom de la société'] = df['Type de poste'].apply(lambda x : x[2] if len(x) > 2 else '')
df['Type de contrat'] = df['Type de poste'].apply(lambda x : x[7].split(' - ')[0] if len(x) > 7 else '')
df.drop(columns='Type de poste', inplace=True)

df = df.apply(lambda c : c.apply(lambda x : ','.join([s.strip("\n ").lower() for s in (x.split(',') if type(x) != list else x)])))

liste_dates = df['Date de publication'].apply(lambda x: '0 j' if x.find('heures') != -1 else x.removeprefix('postée il y a ').replace('postée hier', '1 j').strip('iours'))

df['Date de publication'] = liste_dates.apply(lambda x : pd.to_datetime("2023-01-15") - pd.DateOffset(months=int(x.split()[0])) if x.split()[1] == 'm' else pd.to_datetime("2023-01-15") - pd.DateOffset(days=int(x.split()[0])))
df['Date de publication'] = pd.to_datetime(df['Date de publication'])

df['Salaire minimum'] = liste_salaires.apply(lambda x : float(x.split(' - ')[0].strip(' €\n').replace('.', '').replace(',', '.')) if x != '' else np.nan)
df['Salaire maximum'] = liste_salaires.apply(lambda x : float(x.split(' - ')[1].strip(' €/an\n' + string.ascii_letters).replace('.', '').replace(',', '.')) if x != '' else np.nan)

df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(.*data analyst)", "data analyst", regex=True)
df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(data analyst.*)", "data analyst", regex=True)

df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(.*data ingenieur)", "data ingenieur", regex=True)
df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(data ingenieur.*)", "data ingenieur", regex=True)

df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(.*data scientist)", "data scientist", regex=True)
df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(data scientist.*)", "data scientist", regex=True)

df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(.*chef de projet)", "chef de projet", regex=True)
df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(chef de projet.*)", "chef de projet", regex=True)

df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(.*consultant)", "consultant", regex=True)
df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(consultant.*)", "consultant", regex=True)

df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(.*ingénieur)", "ingénieur", regex=True)
df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(r"(ingénieur.*)", "ingénieur", regex=True)

#print(dates)
df.to_csv("data_test1.csv")
df.to_csv("data_test1.csv", index=False)

In [81]:
df["lieu"] = df["lieu"].where(df["lieu"]=='paris', 'île-de-france')
df.to_csv("data_test2.csv", index=False)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Séparer les données en variables indépendantes (X) et dépendante (y)
X = df[['Intitulé du poste', 'Date de publication', 'lieu', 'competences','Type de poste']]
y = df[['salaire_minimum', 'salaire_maximum']]

# Séparer les données en jeux d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Créer un objet de régression linéaire
reg = LinearRegression()

# Entraîner le modèle sur les données d'entraînement
reg.fit(X_train, y_train)

# Effectuer des prédictions sur les données de test
y_pred = reg.predict(X_test)

# Afficher les résultats de précision
print(reg.score(X_test, y_test))